#### Goal for this webscrape phase: "go into" a page with links to "zoom in"/loop through links that then take you to another level that will then loop through links to scrape each webpage with in
    Challenge:
    -there are 14 styles (categories) and around ~120 links (subcategories)
    -each style can have multiptle pages to loop through

###### Starting small by scraping one page

In [114]:
import cloudscraper
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd  
import time 

In [2]:
url = "https://www.beeradvocate.com/beer/profile/143/1352/" 
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
scraper = cloudscraper.create_scraper() 
info = scraper.get(url) 
print(info.status_code) 
 

200


In [3]:
soup = BeautifulSoup(info.text, 'lxml') 

In [4]:
name=soup.h1.find(string=True)
name

'Shiner Bock'

In [5]:
# soup.find('h1').text
##will give me both the beer title and brewery merged together

In [6]:
# soup.find('dl',{'class':'beerstats'}).text

In [7]:
brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
brewery

'Spoetzl Brewery'

In [9]:
loaction=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
loaction

'Texas'

In [10]:
location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
location2

'United States'

In [11]:
style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
style

'Bock'

In [12]:
ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
ba_score
##official score on ba site 74::'okay'

'74'

In [13]:
over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
over_all_rank
##over all rank is against all beers in ba site

'Ranked #28,056'

In [14]:
style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
style_rank
## style rank is among all beers within style category

'Ranked #96'

In [15]:
alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
alcohol_percent

'4.5%'

In [16]:
avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
avg_rating
# Average across all ratings for this beer

'3.21'

In [17]:
percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
# Percentage of deviation within the ratings

In [18]:
review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
review_count
# Number of reviews for this beer

'1,234'

In [19]:
rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
rating_count
# Number of ratings for this beer

'4,564'

In [20]:
status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
status
# the beer's current activity status

'Active'

In [21]:
date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
date_added
# The date this beer was added to the site

'Oct 03, 1999'

###### zooming out one level to scrape through "bocks" pg 1 with 50 results

In [110]:
# youtube page: John Watson Rooney helped get more of the wabscraping process 
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=0'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website) 
# print(info.status_code)


In [111]:
soup = BeautifulSoup(info.text, 'lxml')

In [24]:
table=soup.find('table')
#table is where the links are concentrated in 

In [25]:
table.find_all("a", href=re.compile("profile"))[::2]

[<a href="/beer/profile/143/1352/"><b>Shiner Bock</b><br/></a>,
 <a href="/beer/profile/35/14309/"><b>Samuel Adams Chocolate Bock</b><br/></a>,
 <a href="/beer/profile/29/1252/"><b>Michelob AmberBock</b><br/></a>,
 <a href="/beer/profile/35/68000/"><b>Samuel Adams Cherry Chocolate Bock</b><br/></a>,
 <a href="/beer/profile/590/2565/"><b>Uff-da</b><br/></a>,
 <a href="/beer/profile/35/74579/"><b>Samuel Adams The Vixen</b><br/></a>,
 <a href="/beer/profile/590/62671/"><b>Back Forty Bock</b><br/></a>,
 <a href="/beer/profile/125/2272/"><b>St. Nikolaus Bock Bier</b><br/></a>,
 <a href="/beer/profile/337/1070/"><b>Spring Bock</b><br/></a>,
 <a href="/beer/profile/2137/3835/"><b>Pandora's Bock</b><br/></a>,
 <a href="/beer/profile/302/41422/"><b>Genesee Spring Bock</b><br/></a>,
 <a href="/beer/profile/66/1873/"><b>Ur-Bock Dunkel</b><br/></a>,
 <a href="/beer/profile/184/20299/"><b>La Trappe Bockbier</b><br/></a>,
 <a href="/beer/profile/35/520715/"><b>Samuel Adams Winter Lager</b><br/></a>,

In [27]:
# links=[link['href'] for link in table.find_all("a", href=re.compile("profile"))[::2]]
# len(links)

50

In [36]:
beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])


In [40]:
beerlinks

['https://www.beeradvocate.com/beer/profile/143/1352/',
 'https://www.beeradvocate.com/beer/profile/35/14309/',
 'https://www.beeradvocate.com/beer/profile/29/1252/',
 'https://www.beeradvocate.com/beer/profile/35/68000/',
 'https://www.beeradvocate.com/beer/profile/590/2565/',
 'https://www.beeradvocate.com/beer/profile/35/74579/',
 'https://www.beeradvocate.com/beer/profile/590/62671/',
 'https://www.beeradvocate.com/beer/profile/125/2272/',
 'https://www.beeradvocate.com/beer/profile/337/1070/',
 'https://www.beeradvocate.com/beer/profile/2137/3835/',
 'https://www.beeradvocate.com/beer/profile/302/41422/',
 'https://www.beeradvocate.com/beer/profile/66/1873/',
 'https://www.beeradvocate.com/beer/profile/184/20299/',
 'https://www.beeradvocate.com/beer/profile/35/520715/',
 'https://www.beeradvocate.com/beer/profile/428/14899/',
 'https://www.beeradvocate.com/beer/profile/417/47795/',
 'https://www.beeradvocate.com/beer/profile/57/7714/',
 'https://www.beeradvocate.com/beer/profile/

##### Need to loop through pages now, as the above only has info for one page which contains 50 links 
-bock category has around 35 pages
-/?sort=revsD&start=1700 (last page of bocks)
-/?sort=revsD&start=0 (first page of bocks)

the number at the end is the only change, however each style has a different number of results and 1700 will not be consistent

maybe there is a way to count down and some how have that be the range when paginating through pages

In [65]:
# testlink='https://www.beeradvocate.com/beer/profile/143/1352/'


beerlist=[]
for x in range(0,)    
    root='https://www.beeradvocate.com'
    header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
    website=f'{root}/beer/styles/32/?sort=revsD&start={x}'
    scraper = cloudscraper.create_scraper() 
    info = scraper.get(website)     
    soup = BeautifulSoup(info.text, 'lxml')
    table=soup.find('table')
    
    beerlinks=[]
    for link in table.find_all("a", href=re.compile("profile"))[::2]:
        beerlinks.append(root + link['href'])
    
    
        for link in beerlinks:
            r=requests.get(link, headers=header)
            soup=BeautifulSoup(r.content,'lxml')
            try:
                name=soup.h1.find(string=True)
            except:
                name= 'Na'
            try:    
                brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
            except:
                brewery= 'Na'
            try:
                location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
            except:
                location= 'Na'
            try:
                location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
            except:
                location2= 'Na'
            try:
                style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
            except:
                style= 'Na'
            try:
                ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
            except:
                ba_score= 'Na'
            try:
                over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
            except:
                over_all_rank= 'Na'
            try:
                style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
            except:
                style_rank= 'Na'
            try:
                alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
            except:
                alcohol_percent= 'Na'
            try:
                avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
            except:
                avg_rating= 'Na'
            try:
                percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
            except:
                percent_dev= 'Na'
            try:
                review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
            except:
                review_count= 'Na'
            try:
                rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
            except:
                rating_count= 'Na'
            try:
                status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
            except:
                status= 'Na'
            try:
                date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
            except:
                date_added= 'Na'
            beer_info= {
                'name': name,
                'brewery': brewery,
                'location':location,
                'location2':location2,
                'style': style,
                'ba_score':ba_score,
                'over_all_rank': over_all_rank,
                'style_rank': style_rank,
                'alcohol_percent': alcohol_percent,
                'avg_rating':avg_rating,
                'percent_dev': percent_dev,
                'review_count': review_count,
                'rating_count':rating_count,
                'status': status,
                'date_added': date_added
                }
            time.sleep(5)
            beerlist.append(beer_info)

In [66]:
# print(beerlist)

[{'name': 'Shiner Bock', 'brewery': 'Spoetzl Brewery', 'location': 'Texas', 'location2': 'United States', 'style': 'Bock', 'ba_score': '74', 'over_all_rank': 'Ranked #28,056', 'style_rank': 'Ranked #96', 'alcohol_percent': '4.5%', 'avg_rating': '3.21', 'percent_dev': '19.63%', 'review_count': '1,234', 'rating_count': '4,564', 'status': 'Active', 'date_added': 'Oct 03, 1999'}, {'name': 'Samuel Adams Chocolate Bock', 'brewery': 'Boston Beer Company (Samuel Adams)', 'location': 'Massachusetts', 'location2': 'United States', 'style': 'Bock', 'ba_score': '82', 'over_all_rank': 'Ranked #25,002', 'style_rank': 'Ranked #70', 'alcohol_percent': '5.8%', 'avg_rating': '3.64', 'percent_dev': '16.21%', 'review_count': '1,247', 'rating_count': '3,144', 'status': 'Active', 'date_added': 'Nov 20, 2003'}, {'name': 'Michelob AmberBock', 'brewery': 'Anheuser-Busch', 'location': 'Missouri', 'location2': 'United States', 'style': 'Bock', 'ba_score': '68', 'over_all_rank': 'Ranked #28,558', 'style_rank': 'R

In [105]:
# df=pd.DataFrame(beerlist)
# df.head()

,name,brewery,location,location2,style,ba_score,over_all_rank,style_rank,alcohol_percent,avg_rating,percent_dev,review_count,rating_count,status,date_added
0,Shiner Bock,Spoetzl Brewery,Texas,United States,Bock,74,"Ranked #28,056",Ranked #96,4.5%,3.21,19.63%,"1,234","4,564",Active,"Oct 03, 1999"
1,Samuel Adams Chocolate Bock,Boston Beer Company (Samuel Adams),Massachusetts,United States,Bock,82,"Ranked #25,002",Ranked #70,5.8%,3.64,16.21%,"1,247","3,144",Active,"Nov 20, 2003"
2,Michelob AmberBock,Anheuser-Busch,Missouri,United States,Bock,68,"Ranked #28,558",Ranked #100,5.1%,2.91,22.34%,732,"1,970",Active,"Aug 17, 2001"
3,Samuel Adams Cherry Chocolate Bock,Boston Beer Company (Samuel Adams),Massachusetts,United States,Bock,79,"Ranked #26,984",Ranked #85,5.8%,3.47,19.6%,188,875,Active,"Apr 11, 2011"
4,Uff-da,New Glarus Brewing Company,Wisconsin,United States,Bock,86,"Ranked #19,084",Ranked #27,6.7%,3.83,10.97%,342,662,Active,"Jan 16, 2002"
